# Results

So the first step is to obtain the project, and map each run to exactly which model, on which dataset, which fold

In [31]:
import os, sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

In [32]:
from warnings import warn

In [33]:
import pandas as pd

In [34]:
from pykt.utils.wandb_utils import WandbUtils

In [35]:
wu = WandbUtils('letech', 'kt_toolkits')
wu.sweep_dict

Warning!! sweep falconcode_2_3_akt_qid_4, was ran multiple times, taking the latest one :['xyw3chqc', '0iuq5zge']
Warning!! sweep falconcode_2_3_akt_qid_3, was ran multiple times, taking the latest one :['isgm2mjl', 'df07rq0a']
Warning!! sweep falconcode_2_3_akt_qid_2, was ran multiple times, taking the latest one :['ha86l3dj', 'azqredp7']
Warning!! sweep falconcode_2_3_akt_qid_1, was ran multiple times, taking the latest one :['hzcqe9uk', 'hteu3v8z']
Warning!! sweep falconcode_2_3_akt_qid_0, was ran multiple times, taking the latest one :['xyoh1w47', 'gf367wp3']
Warning!! sweep falconcode_2_3_dkvmn_qid_4, was ran multiple times, taking the latest one :['wft4n7v0', '9ysx4g5k', 'tr0ehscq']
Warning!! sweep falconcode_2_3_dkvmn_qid_3, was ran multiple times, taking the latest one :['lx8w0ov9', 'kizckc34']
Warning!! sweep falconcode_2_3_dkvmn_qid_2, was ran multiple times, taking the latest one :['3h0cbp34', '9apg2mht']
Warning!! sweep falconcode_2_3_dkvmn_qid_1, was ran multiple times, ta

{'falconcode_2_3_akt_qid_4': 'xyw3chqc',
 'falconcode_2_3_akt_qid_3': 'isgm2mjl',
 'falconcode_2_3_akt_qid_2': 'ha86l3dj',
 'falconcode_2_3_akt_qid_1': 'hzcqe9uk',
 'falconcode_2_3_akt_qid_0': 'xyoh1w47',
 'falconcode_2_3_dkvmn_qid_4': 'wft4n7v0',
 'falconcode_2_3_dkvmn_qid_3': 'lx8w0ov9',
 'falconcode_2_3_dkvmn_qid_2': '3h0cbp34',
 'falconcode_2_3_dkvmn_qid_1': 'mmx5w69y',
 'falconcode_2_3_dkvmn_qid_0': 'xkad2i1b',
 'falconcode_2_3_dkt_qid_4': '6cqeht5u',
 'falconcode_2_3_dkt_qid_3': 'jznnp2u3',
 'falconcode_2_3_dkt_qid_2': 'scpgb7cb',
 'falconcode_2_3_dkt_qid_1': '0zbdr44q',
 'falconcode_2_3_dkt_qid_0': 'ac8jsdnf',
 'falconcode_2_2_akt_qid_4': 'i6g8hkul',
 'falconcode_2_2_akt_qid_3': 'jar75181',
 'falconcode_2_2_akt_qid_2': '9pb079ex',
 'falconcode_2_2_akt_qid_1': 'rfs54326',
 'falconcode_2_2_akt_qid_0': '1vm4shz0',
 'falconcode_2_2_dkvmn_qid_4': '4it4rpcs',
 'falconcode_2_2_dkvmn_qid_3': 'uljidmwb',
 'falconcode_2_2_dkvmn_qid_2': 'crrz4438',
 'falconcode_2_2_dkvmn_qid_1': 'rexz7xll'

#### We obtain a list of the datasets we used and the models trained

In [36]:
datasets, models = set(), set()
sweep_names = wu.sweep_dict.keys()
for sn in sweep_names:
    parts = sn.split("_")
    datasets.add("_".join(parts[:3]))
    models.add("_".join(parts[3: parts.index("qid")]))
    
datasets, models

({'falconcode_23_4', 'falconcode_2_2', 'falconcode_2_3'},
 {'akt', 'dkt', 'dkvmn'})

#### Obtain for the given dataset, model, and folds, the best results

We also need to know for that given model, what are the hyperparameters, so w

In [37]:
def get_hyperparameter_names(selected_sweeps):
    _, model_configs = wu.get_df(selected_sweeps[0])
    rejected_paramters = ["model_name", "dataset_name", "save_dir", "fold", "seed"]
    hyperparameters = [k for k in model_configs if k not in rejected_paramters]
    return hyperparameters

In [38]:
def get_best_hyperparameters(grouped_results, hyperparams):
    best_hyperparams = grouped_results.groupby(hyperparams)["validauc"].mean().to_frame()
    # Obtain the dataframe with the order of importance of each hyperparameter combination
    best_hyperparams = best_hyperparams.sort_values(by="validauc", ascending=False)
    print(best_hyperparams)
    best_hyperparams = best_hyperparams.head(1).index
    best_hyperparams = best_hyperparams.to_frame().reset_index(drop=True).iloc[0].to_dict()
    query_best_hyperparams = " & ".join([f"({k}=={repr(v)})" for k, v in best_hyperparams.items()])
    return best_hyperparams, query_best_hyperparams

In [39]:
def check_all_finnished(selected_sweeps):
    for ss in selected_sweeps:
        status = wu.get_sweep_info(ss)["state"]
        if status != "FINISHED":
            message = "Careful sweep " + ss + " hasn't finnished yet"
            warn(message)

In [40]:
from collections import defaultdict
from itertools import product

f = lambda : defaultdict(dict)
dataframes = []
best_hyperparams = defaultdict(f)
for dataset_name, model_name in product(datasets, models):
    selected_sweeps = [sn for sn in sweep_names 
                       if dataset_name in sn and model_name in sn]
    check_all_finnished(selected_sweeps)
    grouped_results = pd.concat(wu.get_multi_df(selected_sweeps))
    hyperparameters = get_hyperparameter_names(selected_sweeps)
    bests, query = get_best_hyperparameters(grouped_results, 
                                     hyperparameters)
    best_hyperparams[dataset_name][model_name] = bests
    dataframes.append(grouped_results.query(query))
    
best_hyperparams = dict(best_hyperparams)
best_hyperparams

/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_akt_qid_4 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_akt_qid_3 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_akt_qid_2 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_akt_qid_1 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_akt_qid_0 hasn't finnished yet
  warn(message)


                                                                     validauc
d_ff d_model dropout emb_type n_blocks learning_rate num_attn_heads          
256  256     0.05    qid      4        0.00010       8               0.992845
64   256     0.10    qid      4        0.00010       4               0.992831
256  256     0.10    qid      1        0.00100       8               0.992642
     64      0.05    qid      4        0.00100       8               0.992617
             0.10    qid      4        0.00100       8               0.992608
...                                                                       ...
             0.30    qid      4        0.00001       8               0.987595
64   64      0.50    qid      1        0.00001       8               0.987530
256  256     0.50    qid      1        0.00001       4               0.987240
     64      0.50    qid      1        0.00010       8               0.987056
64   256     0.50    qid      4        0.00001       4          

/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_dkvmn_qid_4 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_dkvmn_qid_3 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_dkvmn_qid_2 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_dkvmn_qid_1 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_dkvmn_qid_0 hasn't finnished yet
  warn(message)


                                             validauc
dim_s size_m dropout emb_type learning_rate          
64    64     0.10    qid      0.00100        0.991180
      32     0.05    qid      0.00100        0.991074
             0.10    qid      0.00100        0.991066
      64     0.05    qid      0.00100        0.991004
      32     0.05    qid      0.00010        0.988739
             0.10    qid      0.00010        0.988712
      64     0.10    qid      0.00010        0.988653
             0.05    qid      0.00010        0.988455
      32     0.05    qid      0.00001        0.981631
      64     0.05    qid      0.00001        0.974023
                                         validauc
dropout emb_size emb_type learning_rate          
0.05    256      qid      0.00100        0.992152
0.10    256      qid      0.00100        0.992144
0.30    256      qid      0.00100        0.992087
0.50    256      qid      0.00100        0.992021
0.05    64       qid      0.00100        0.991190
0.

/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_3_akt_qid_4 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_3_akt_qid_3 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_3_akt_qid_2 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_3_akt_qid_1 hasn't finnished yet
  warn(message)
/tmp/ipykernel_2329/1632450084.py:6: UserWarning: Careful sweep falconcode_2_3_akt_qid_0 hasn't finnished yet
  warn(message)


KeyError: '_timestamp'

In [ ]:
def get_best_models(grouped_results):
    f = lambda df: df.sort_values(by="validauc").iloc[-1]
    return grouped_results.groupby("fold").apply(f)

In [ ]:
from collections import defaultdict
from itertools import product

f = lambda : defaultdict(dict)
dataframes = []
best_models = defaultdict(f)
for dataset_name, model_name in product(datasets, models):
    selected_sweeps = [sn for sn in sweep_names 
                       if dataset_name in sn and model_name in sn]
    check_all_finnished(selected_sweeps)
    grouped_results = pd.concat(wu.get_multi_df(selected_sweeps))
    sub_df = get_best_models(grouped_results)
    best_models[dataset_name][model_name] = list(sub_df.model_save_path)

In [ ]:
best_models['falconcode_2_2']['dkt']

At the end here we have a mapping of the hyperparameters to rerun model training with, which we will save somewhere in a file or a dictionary. We can then pass this dictionary to another training script which will run the model trainings with the good hyperparameters

In [ ]:
import json

best_configs_path = "/home/koutchc1/pykt-toolkit/configs/best_model.json"

data = {}
if os.path.exists(best_configs_path):
    with open(best_configs_path, "r") as f:
        data = json.load(f)
data.update(best_models)

with open(best_configs_path, "w") as f:
    json.dump(data, f, indent = 3)

In [ ]:
best_models_txt_path = "/home/koutchc1/pykt-toolkit/best_models.txt"
with open(best_models_txt_path, 'w') as f:
    for dataset, models_to_path in data.items():
        for paths in models_to_path.values():
            for path in paths:
                f.write("/".join(path.split("/")[:-1]) + "\n")